### Eksik Değer Problemini Çözme

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
def load():
    data = pd.read_csv("titanic.csv")
    return data

df = load()
df.headad()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def missing_values_table(dataframe, na_name=False):
    
    # Sadece eksikliğe sahip kolonların (değişkenlerin) seçilmesi:
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    # Eksik değer sayısı:
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    
    # Eksik değer oranı:
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    
    # bunları bir dataframe'e çevirme:
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns

missing_values_table(df, True)

          n_miss  ratio
Cabin        687  77.10
Age          177  19.87
Embarked       2   0.22


['Age', 'Cabin', 'Embarked']

* DİKKAT: Eğer ağaca dayalı yöntemler kullanılıyorsa bu durumda eksik değerler tıpkı aykırı değerlerdeki gibi etkisi göz ardı edilebilir durumlardır. Bunun sebebi bir makine öğrenmesi eğitimi kapsamında daha detaylı bir şekilde ele alınacaktır. Fakat şu anda bir Feature engineering eğitimindeyiz. Özetle ifade etmek gerekirse ağaç yöntemleri doğrusal yöntemlerde ya da gradient descent gibi optimizasyona dayalı yöntemlerde olduğu gibi değil de daha esnek ve dallara ayırmalı bir şekilde çalışıyor olduğundan dolayı bu noktadaki aykırıklıklar ve eksikliklerin etkisi neredeyse yoka yakındır. Tamamen etkisi yok demek doğru olmayabilir ama yoka yakındır. Yani göz ardı etmek mantıklıdır. Bir istisnası var eğer ilgilendiğimiz problem regresyon problemi ise ve bağımlı değişken de dolayısıyla sayısal bir değişkense bu durumda orada aykırılık olması durumunda sonuca gitme süresi biraz uzayabilir eğer gradient descent temelli yine bir optimizasyon yöntemi ile bir ağaç yöntemi kullanılıyorsa bu durumda sadece bu exception vardır. Oradaki bağımlı değişkende yer alabilecek olan aykırıklıklardan dolayı optimizasyon işlemlerinin birazcık süresi uzayabilir. Toparlayacak olursak özetle eksik değer ve aykırı değer problemlerine genel hatları itibari ile çok iyi hakim olmak lazım ama nerede ne kadar etkilerini olduğunu da iyi bilmek lazım. Doğrusal yöntemlerde ve gradient descent temelli yöntemlerde bizim için bu teknikler çok daha hassas iken ağaca dayalı yöntemlerde bulunan etkisi çok çok daha düşüktür.

###### Çözüm 1: Hızlıca Silmek

In [5]:
df.dropna().shape
# 891 tane gözlem vardı 183 tane kaldı çünkü bir satırda en az 1 tane bile eksik değer varsa dropna onları uçuruyor olacaktır. 
# Dolayısıyla dropna kullanırken buna dikkat ediyor olmaz lazım. Elimizdeki veri setinde eksik değer problemimiz varsa ve eğer 
# gözlem sayımız çok fazlaysa mesela 500000 1000000 3000000 vs. gibi bu durumda direkt silmek tercih edilebilir. Veri boyutunun 
# yeterli olmasından dolayı. 

(183, 12)

###### Çözüm 2: Basit Atama Yöntemleri ile Doldurmak

In [6]:
# Örneğin yaştaki eksiklikleri ortalaması veya medyanıyla doldurabileceğimiz gibi herhangi sabit bir değerle de doldurabiliriz.
df["Age"].fillna(df["Age"].mean())
# Bunu gözlemleyemiyoruz ama çünkü yeniden atama işlemi yapmadık.

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [7]:
# Bu şekilde gözlemleyebiliriz:
df["Age"].fillna(df["Age"].mean()).isnull().sum()

# Eksiklikler dolmuş gibi gözüküyor...

0

In [8]:
# Median değerri ile dolduralım:
df["Age"].fillna(df["Age"].mean()).isnull().sum()

0

In [9]:
# Sabit sayı ile dolduralım:
df["Age"].fillna(0).isnull().sum()

0

* DİKKAT: Veri setinde bir sürü değişken var biz bunları nasıl hızlı bir şekilde dolduracağız? Yani biz burada yaş değişkenini gördük ve yaptık ancak elimizde birçok değişken olursa ne yapacağız? Biraz daha bu işlemlerimizi ileri seviyeye taşıyalım:

In [11]:
# Daha önce öğrendiğimiz lambda ve fillna'i kullanarak bunları dolduralım düşüncemizin olduğunu düşünelim:
df.apply(lambda x: x.fillna(x.mean()), axis=0)

TypeError: Could not convert Braund, Mr. Owen HarrisCumings, Mrs. John Bradley (Florence Briggs Thayer)Heikkinen, Miss. LainaFutrelle, Mrs. Jacques Heath (Lily May Peel)Allen, Mr. William HenryMoran, Mr. JamesMcCarthy, Mr. Timothy JPalsson, Master. Gosta LeonardJohnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)Nasser, Mrs. Nicholas (Adele Achem)Sandstrom, Miss. Marguerite RutBonnell, Miss. ElizabethSaundercock, Mr. William HenryAndersson, Mr. Anders JohanVestrom, Miss. Hulda Amanda AdolfinaHewlett, Mrs. (Mary D Kingcome) Rice, Master. EugeneWilliams, Mr. Charles EugeneVander Planke, Mrs. Julius (Emelia Maria Vandemoortele)Masselmani, Mrs. FatimaFynney, Mr. Joseph JBeesley, Mr. LawrenceMcGowan, Miss. Anna "Annie"Sloper, Mr. William ThompsonPalsson, Miss. Torborg DaniraAsplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)Emir, Mr. Farred ChehabFortune, Mr. Charles AlexanderO'Dwyer, Miss. Ellen "Nellie"Todoroff, Mr. LalioUruchurtu, Don. Manuel ESpencer, Mrs. William Augustus (Marie Eug

* Satırlara göre gitmesi için axis = 0 dedik. Satırlara göre gittiğinde aslında sütun bazında aslında konuya bakmış oluyoruz. Dolayısıyla konuya sütun bazında satırlar açısından bakmayı axis = 0 diyerek ifade ediyoruz, normalde axis = 0 satırları 1 sütunları ifade eder ama buradaki odağımız aşağıya doğru tüm satırlara bakmaktır. Yani örneğin satırın ortalamasını alacağız, sütunun ortalamasını alacağız, ama sütunun ortalamasını almak demek aşağıya doğru satırlara odaklan demektir bundan dolayı axis = 0 yapıyoruz.
* Object tipi ortalamasını mı alacaksın diye bize kızıp hata verdi. Şimdi bu veri setinin içerisinde hem sayısal değişkenler var hem kategorik değişkenler var öyle bir işlem yapmalıyız ki hayat kurtaran serisinden buradaki sadece sayısal değişkenleri doldurmalıyız.
* DİKKAT: Bakın napıyoruz şimdi? fillna bölümü yine doldurma işlemi... lambdamız ve applyımız var... Diyoruz ki eğer ilgili değişkenin tipi objectten farklı ise bu değişkeni ortalaması ile doldur, eğer ilgili değişkenin tipi objectten farklı değilse olduğu gibi kalsın diyoruz.

In [12]:
df.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
# Bunun sonucunu kaydedelim ve üzerinden ilerleyelim:
dff = df.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0)

In [16]:
# isnull fonksiyonumuzu bir çağıralım:
dff.isnull().sum().sort_values(ascending = False)

Cabin          687
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

* Sayısal değişkendeki (Sadece yaş değişkeninde eksiklik vardı) bunlardan kurtulduk. İyi de buradaki cabin ve embarked değişkenleri ne olacak? Yani kategorik değişkenlerdeki eksikliklere ne yapacağız?
* DİKKAT: Kategorik değişkenler için en mantıklı görülebilecek doldurma yöntemlerinden birisi modunu almaktır. Yani buradaki embarked değişkenini doldurmak için embarked değişkeninin modunu alabiliriz. Bu modun string karşılığına yani value'suna erişmek için 0.indeksi getir dediğimizde bakalım ne getirecek:

In [18]:
df["Embarked"].fillna(df["Embarked"].mode()[0])

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [19]:
df["Embarked"].fillna(df["Embarked"].mode()[0]).isnull().sum()

0

In [20]:
# Eğer istersek buradaki herhangi özel bir ifadeyi de kullanabiliriz, mesela embarked'daki eksiklikleri string bir missing 
# ifadesi ile doldur diyebiliriz:
df["Embarked"].fillna("missing").isnull().sum()

0

In [27]:
# Bunu yukarıda yaptığımız gibi otomatik olarak nasıl yaparız?
df.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= 10) else x, axis = 0)

# len(x.unique()) <= 10 ifadesi ile eşsiz değer sayısı <= 10 olanları ifade ediyoruz. Örneğin cinsiyet, embarked değişkeni...

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
